In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()
df = spark.read.csv('avocado.csv', sep=',', header=True, inferSchema=True,
                         nullValue='NA')
df.show(5)

- Ganti Data menjadi VG Sales dengan feature = JP, NA, EU, Global
- Label Platform atau Genre (salah satu saja)

In [ ]:
df.toPandas()

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='region', outputCol='label')
indexer_model = indexer.fit(df)
df_indexed = indexer_model.transform(df)

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Small Bags','Large Bags','XLarge Bags'],
                            outputCol='features')
df_assembled = assembler.transform(df_indexed)
df_assembled.select('features', 'label').show(5, truncate=False)

In [ ]:
df_train, df_test = df_assembled.randomSplit([0.8, 0.2], seed=35)
training_ratio = df_train.count() / df_assembled.count()
print(training_ratio)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree_model = tree.fit(df_train)
prediction = tree_model.transform(df_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

In [ ]:
prediction.select('label', 'prediction', 'probability').toPandas()

In [ ]:
TP = prediction.filter('label = prediction').count()
FP = prediction.filter('label != prediction').count()
accuracy = TP / (TP+FP)
print("True Positive ",TP)
print("False Positive ",FP)
print("Accuracy ",accuracy)